In [ ]:
from inference_utils.cam_predictor import CamPredictorMultiLabel
from pathlib import Path

import torch
from omegaconf import OmegaConf

from modules.classification.efficient_net_v2_module import EfficientNetV2Module
from modules.classification.resnet_module import ResNetModule

# 00006160_001.png
# 00014753_000.png
# 00013613_007.png good example of shitty image
# 00028730_000.png no finding

model_path = Path('../lightning_logs/resnet34_epoch=17_val_auroc=0.843_top_updated.pt')
dataset_path = Path('/home/szymswiat/datasets/nih_dataset')
img_size = 384

state = torch.load(model_path.as_posix())
hparams = OmegaConf.create(state['hparams'])
thresholds = torch.tensor(state['thresholds'])
classes = OmegaConf.to_object(hparams.dynamic.classes)

if hparams.architecture == 'eff_net_v2':
    model_class = EfficientNetV2Module
elif hparams.architecture == 'resnet':
    model_class = ResNetModule
else:
    raise ValueError()

model = model_class.create_from_state(state)
model.eval()

gradcam_target_layer = model.model.layer4[0]
cam_pred = CamPredictorMultiLabel(model, thresholds, gradcam_target_layer)

In [ ]:
import torch
from pathlib import Path

model_dir = Path('models')

x = torch.onnx.export(model=cam_pred,
                      f=(model_dir / 'resnet_cam.onnx').as_posix(),
                      args=(torch.zeros(3, 384, 384, requires_grad=True), ),
                      opset_version=11)